In [91]:
import re
import gradio as gr
from itertools import count
import numpy as np
# add imports here
import pandas as pd
from scipy.constants import horsepower

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from streamlit import title, button

In [92]:
# add from here
from sklearn.preprocessing import LabelEncoder

In [93]:
train_data = pd.read_csv('data/train.csv')
train_data.drop(columns=['id'], inplace=True)
train_data.head(5)


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [94]:
# # Getting rid of the outliers
# # Getting rid of the cars that are more expensive than 120000. More than this in usd is more of an outlier for normal people
# train_data = train_data[train_data.price < 120000]
# # Getting rid of the cars that are older than 1995. This is a bit arbitrary, but it's a start. There is also a bigger likelihood for outliers in the older cars
# train_data = train_data[train_data.model_year > 1995]
## Didn't work as intended

In [95]:
# Fuel type
# Hybrid
# Due to dual engines you could assume they cost more
fuel_mapping = {
    'Hybrid': ['hybrid'],
    'Electric': ['electric'],
    'Diesel': ['diesel'],
    'E85': ['e85'],
    'Gasoline': ['gasoline']
}


# Function to map fuel types
def map_fuel_type(value):
    value_lower = value.lower()
    for fuel, keywords in fuel_mapping.items():
        if any(keyword in value_lower for keyword in keywords):
            return fuel
    return 'Other'


# Apply the mapping function
train_data['fuel_type'] = train_data['fuel_type'].astype(str).apply(map_fuel_type)

# print(train_data)
train_data.head(5)


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [96]:
# may want to add a more ideal way to fix the color hell
# I wonder if there is an external library that can help with this or I have to do it manually
# Extract color information
color_mapping = {
    'Blue': ['blue', 'navy', 'sea', 'glacier', 'sapphire', 'midnight'],
    'Black': ['black', 'onyx', 'obsidian', 'jet', 'ebony'],
    'White': ['white', 'pearl', 'ivory', 'alabaster', 'cream', 'frost'],
    'Silver': ['silver', 'platinum', 'titanium', 'steel', 'graphite', 'ash'],
    'Gray': ['gray', 'grey', 'charcoal', 'slate', 'gunmetal'],
    'Red': ['red', 'crimson', 'ruby', 'cherry', 'garnet'],
    'Green': ['green', 'emerald', 'olive', 'lime', 'forest'],
    'Brown': ['brown', 'copper', 'bronze', 'mahogany', 'rust'],
    'Yellow': ['yellow', 'gold', 'amber', 'honey', 'sun'],
    'Orange': ['orange', 'copper', 'bronze', 'mahogany', 'rust'],
    'Purple': ['purple', 'violet', 'lavender', 'plum', 'lilac'],
    'Pink': ['pink', 'rose', 'fuchsia', 'blush', 'coral'],
    'Beige': ['beige', 'tan', 'cream', 'sand', 'camel'],
    'Gold': ['gold', 'yellow', 'amber', 'honey', 'sun'],
    'Bronze': ['bronze', 'brown', 'copper', 'mahogany', 'rust']
}


# Function to map colors
def map_color(value):
    value_lower = value.lower()
    for color, keywords in color_mapping.items():
        if any(keyword in value_lower for keyword in keywords):
            return color
    return value


# Apply the mapping function
train_data['ext_col'] = train_data['ext_col'].astype(str).apply(map_color)

# print(train_data)


In [97]:
# Accident and clean title
# Accident

# Convert accident column: 'At least 1' -> 1, else -> 0
train_data['accident'] = train_data['accident'].apply(lambda x: 1 if 'At least 1' in str(x) else 0)

# Convert clean_title column: 'Yes' -> 1, else -> 0
train_data['clean_title'] = train_data['clean_title'].apply(lambda x: 1 if 'Yes' in str(x) else 0)

train_data.accident = train_data.accident.astype(str).apply(lambda x: 1 if 'At least 1' in x else 0)

# Clean title
train_data.clean_title = train_data.clean_title.astype(str).apply(lambda x: 1 if 'Yes' in x else 0)


In [98]:
# Transmission
# Simplify transmission to 'Manual' or 'Automatic'

def categorize_transmission(x):
    x = str(x).lower()  # Convert to lowercase for consistent matching
    if 'manual' in x or 'm/t' in x:
        return 'Manual'
    elif 'automatic' in x or 'a/t' in x:
        return 'Automatic'
    else:
        return 'Other'

# Apply the categorization function
train_data['transmission'] = train_data['transmission'].apply(categorize_transmission)

In [99]:
# For use in gradio
gr_data = train_data.copy()

In [100]:
# Extract engine information


# Extract horsepower(HP) from the 'engine' column now kw
def extract_engine(df):
    #as a non car person i would assume that cylinders are not as important as the horsepower and the engine size for it's value
    if pd.isnull(df):
        return np.nan
    hp_match = re.search(r'(\d+\.?\d*)HP', df)  # HP hp = re.search(r'(\d+\.?\d*)HP', df) #litt vel case sensitive. Siden jeg ikke sjekker om det er hP eller Hp ellr hp. Oppdatering anbefales
    engine_size_match = re.search(r'(\d+\.?\d*)\s*[lL]', df)  # L, test if it works like it is
    cylynder_match = re.search(r'(\d+\.?\d*)\s*[cC][yY][lL]', df)  # Cylinders

    # Extract and return the horsepower and engine size as a float
    horsepower = float(hp_match.group(
        1)) if hp_match else None  #np.nan # May need to rename the horsepower variable to something like hp. Since horsepower is from the scipy.constants
    engine_size = float(engine_size_match.group(1)) if engine_size_match else None  #np.nan
    cylynder = float(cylynder_match.group(1)) if cylynder_match else None

    return horsepower, engine_size, cylynder
    # if pd.isnull(df):
    #     return np.nan
    # df = str(df)
    # hp = re.search(r'(\d+\.?\d*)\s*[hH][pP]', df)
    # return float(hp.group(1)) if hp else np.nan


# Apply the refined horsepower extraction function to the 'engine' column
train_data[['horsepower', 'engine_size', 'cylynder']] = train_data['engine'].apply(
    lambda x: pd.Series(extract_engine(x)))

train_data.drop(columns=['engine'], inplace=True)

train_data.horsepower = train_data.horsepower.fillna(0)
train_data.engine_size = train_data.engine_size.fillna(0)
train_data.cylynder = train_data.cylynder.fillna(0)

In [101]:
#int_col
# Extract color information

# Function to map colors
# Define the interior color mapping dictionary for car interiors
interior_color_mapping = {
    'Black': ['black', 'onyx', 'ebony', 'charcoal'],
    'Gray': ['gray', 'grey', 'ash', 'slate', 'pewter'],
    'Beige': ['beige', 'tan', 'cream', 'sand', 'camel'],
    'Brown': ['brown', 'chocolate', 'espresso', 'cocoa', 'saddle'],
    'White': ['white', 'ivory', 'alabaster', 'frost', 'pearl'],
    'Red': ['red', 'crimson', 'burgundy', 'wine', 'maroon'],
    'Blue': ['blue', 'navy'],
    'Green': ['green', 'olive'],
    # Add other interior-specific colors or materials as needed
}


# Function to map interior colors
def map_interior_color(value):
    # Convert the value to lowercase for case-insensitive matching
    value_lower = value.lower()
    # Iterate over the color mapping dictionary
    for color, keywords in interior_color_mapping.items():
        # Check if any keyword for the color is present in the value
        if any(keyword in value_lower for keyword in keywords):
            return color
    return 'Other'


# Apply the mapping function to the 'interior_color' column
train_data['int_col'] = train_data['int_col'].astype(str).apply(map_interior_color)



In [102]:
# Label encoding

# Define the categorical columns
label_columns = ['brand', 'model', 'fuel_type', 'ext_col', 'int_col', 'transmission']
# Initialize label encoders
label_encoders = {col: LabelEncoder() for col in label_columns}

# Apply label encoding to the categorical columns
for col in label_columns:
    train_data[col] = label_encoders[col].fit_transform(train_data[col])

X = train_data.drop(columns=['price'])
y = train_data['price']

In [103]:
from xgboost import XGBClassifier

#XGBRegressor. Already tested this in a previous notebook, and it worked if i didn't have problems with pipelining the data

# Tips for å velge modell når man ikke vet hvilken ende. Få chatgpt til å gi flere valg man kan velge mellom og prøve ut;)
xgb_model = xgb.XGBRegressor()
# xgb_model = XGBClassifier()

# parameters = {
#     'n_estimators':[10000],
#     'random_state':[42],
#     'subsample':[0.9],
#     'min_child_weight':[10],
#     'max_depth':[6],
#     'learning_rate':[0.01],
#     'gamma':[0.5],
#     'colsample_bytree':[0.9],
#     'lambda':[0],
#     'alpha':[0.1],
#     # 'scale_pos_weight':[len(y[y == 0]) ],
#     # 'objective':['binary:logistic']
# }
parameters = {  # go over the parameters and see if they are correct because these are now automatically generated
    'n_estimators': [100],  #, 500, 1000
    'max_depth': [3],
    'learning_rate': [0.3],
    'subsample': [0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'gamma': [0, 0.1],
    'min_child_weight': [5],
    'lambda': [0, 0.1],
    'alpha': [0.1, 0.2]
}

# Grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=parameters, cv=3, verbose=2,
                           scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X, y)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {-grid_search.best_score_}")

best_xgb_model = grid_search.best_estimator_
best_xgb_model.fit(X, y)

# Predict the target on the training data
y_predict_train = best_xgb_model.predict(X)

# Calculate the Root Mean Squared Error (RMSE) on the training data
rmse_train = np.sqrt(mean_squared_error(y, y_predict_train))
print(f"Root Mean Squared Error on training data: {rmse_train}")


Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END alpha=0.1, colsample_bytree=0.5, gamma=0, lambda=0, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END alpha=0.1, colsample_bytree=0.5, gamma=0, lambda=0, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END alpha=0.1, colsample_bytree=0.5, gamma=0, lambda=0, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END alpha=0.1, colsample_bytree=0.5, gamma=0, lambda=0.1, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END alpha=0.1, colsample_bytree=0.5, gamma=0, lambda=0.1, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END alpha=0.1, colsample_bytree=0.5, gamma=0, lambda=0.1, learning_rate=0.3, max_depth=3, min_child_weigh

Traceback (most recent call last):
  File "C:\Users\sindr\anaconda3\Lib\site-packages\gradio\queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sindr\anaconda3\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sindr\anaconda3\Lib\site-packages\gradio\blocks.py", line 2015, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sindr\anaconda3\Lib\site-packages\gradio\blocks.py", line 1562, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sindr\anaconda3\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
 

[CV] END alpha=0.2, colsample_bytree=0.5, gamma=0.1, lambda=0, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.3s
[CV] END alpha=0.2, colsample_bytree=0.5, gamma=0.1, lambda=0, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.3s
[CV] END alpha=0.2, colsample_bytree=0.5, gamma=0.1, lambda=0, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END alpha=0.2, colsample_bytree=0.5, gamma=0.1, lambda=0.1, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END alpha=0.2, colsample_bytree=0.5, gamma=0.1, lambda=0.1, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END alpha=0.2, colsample_bytree=0.5, gamma=0.1, lambda=0.1, learning_rate=0.3, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.9; total time=

In [104]:
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt
#
# cm = confusion_matrix(y, y_predict_train)
# # sns.heatmap(cm, annot=True, fmt='d')
#
# plt.show()
train_data.head(5)

,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,horsepower,engine_size,cylynder
0,31,495,2007,213000,2,0,53,4,0,0,4200,172.0,1.6,4.0
1,28,930,2002,143250,2,0,48,0,0,0,4999,252.0,3.9,8.0
2,9,1575,2002,136731,1,0,10,4,0,0,13900,320.0,5.3,8.0
3,16,758,2017,19500,2,2,8,1,0,0,45000,420.0,5.0,8.0
4,36,1077,2021,7388,2,0,8,0,0,0,97500,208.0,2.0,4.0


In [105]:
# Sort
def cat_sort(cat):
    count_cat = gr_data[cat].value_counts()
    sorted_cat = list(count_cat.index)
    return sorted_cat


sorted_brand = cat_sort('brand')
sorted_model = cat_sort('model')
sorted_fuel_type = cat_sort('fuel_type')
sorted_ext_col = cat_sort('ext_col')
sorted_int_col = cat_sort('int_col')
sorted_transmission = cat_sort('transmission')


In [106]:
print(gr.__version__)

5.5.0


In [107]:
# Define the function to build a car and predict the price
def car_builder(brand, model, model_year, milage, horsepower, engine_size, cylynder,fuel_type, ext_col, int_col, transmission, accident,
                clean_title):
    # Convert the input values to the corresponding label encoded values
    accident_value = 1 if accident == 'Yes' else 0
    clean_title_value = 1 if clean_title == 'Yes' else 0
    # Convert the input values to the corresponding label encoded values
    tmp = {
        'brand': [label_encoders['brand'].transform([brand])[0]],
        'model': [label_encoders['model'].transform([model])[0]],
        'model_year': [model_year],
        'milage': [milage],
        'horsepower': [extract_engine(horsepower)[0]],
        'engine_size': [extract_engine(engine_size)[1]],
        'cylynder': [extract_engine(cylynder)[2]],
        'fuel_type': [label_encoders['fuel_type'].transform([fuel_type])[0]],
        'ext_col': [label_encoders['ext_col'].transform([ext_col])[0]],
        'int_col': [label_encoders['int_col'].transform([int_col])[0]],
        'transmission': [label_encoders['transmission'].transform([transmission])[0]],
        'accident': [accident_value],
        'clean_title': [clean_title_value]
    }
    df = pd.DataFrame(tmp)

    pricePrediction = best_xgb_model.predict(df)[0].astype(int)

    # Return the predicted price
    return f"Predicted price: {pricePrediction} USD"


# Function to get the current year
from datetime import datetime
def get_current_year():
    return datetime.now().year

# sorted_brand = cat_sort('brand')
# sorted_model = cat_sort('model')
# sorted_fuel_type = cat_sort('fuel_type')
# sorted_ext_col = cat_sort('ext_col')
# sorted_int_col = cat_sort('int_col')
# sorted_transmission = cat_sort('transmission')
# Define sorted options (assuming you've defined sorting functions as in your original code)
sorted_brand = [label_encoders['brand'].inverse_transform([i])[0] for i in range(len(label_encoders['brand'].classes_))]
sorted_model = [label_encoders['model'].inverse_transform([i])[0] for i in range(len(label_encoders['model'].classes_))]
sorted_fuel_type = [label_encoders['fuel_type'].inverse_transform([i])[0] for i in range(len(label_encoders['fuel_type'].classes_))]
sorted_ext_col = [label_encoders['ext_col'].inverse_transform([i])[0] for i in range(len(label_encoders['ext_col'].classes_))]
sorted_int_col = [label_encoders['int_col'].inverse_transform([i])[0] for i in range(len(label_encoders['int_col'].classes_))]
sorted_transmission = [label_encoders['transmission'].inverse_transform([i])[0] for i in range(len(label_encoders['transmission'].classes_))]

demo = gr.Interface(
    fn=car_builder,
    inputs=[
        gr.Dropdown(choices=sorted_brand, value=sorted_brand[0]),
        gr.Dropdown(choices=sorted_model, value=sorted_model[0]),
        gr.Number(value=2024, label="Model Year"),
        gr.Slider(0, 1000000, value=0 ,label="Milage"),
        gr.Number(label="horsepower", value=0),
        gr.Number(label="engine size L", value=0),
        gr.Number(label="cylynder count", value=0),
        gr.Dropdown(choices=sorted_fuel_type, label="Fuel Type", value=sorted_fuel_type[0]),
        gr.Dropdown(choices=sorted_transmission, label="Transmission", value=sorted_transmission[0]),
        gr.Dropdown(choices=sorted_ext_col, label="Exterior Color", value=sorted_ext_col[0]),
        gr.Dropdown(choices=sorted_int_col, label="Interior Color", value=sorted_int_col[0]),
        gr.Radio(['Yes', 'No'], label="Accident"),
        gr.Radio(['Yes', 'No'], label="Clean Title"),

    ],
    outputs='text',
    title='Car Price Predictor',
    description='Predict the price of a car based on its features.'
)
# def close_interface():
#     demo.close()
#
# with gr.Row():
#     close_button = gr.Button("Close the demo")
#     # close_button.onClick(close_interface)
#     # close_button.click(close_interface())

#main
if __name__ == '__main__':
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7868

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [108]:
# make a closing statement or condition
# For now just uncomment the code under and run it:

# Need to make a button so I can use demo close

# button("Close the demo")
# if button:
#     demo.close()
# demo.close()
